In [1]:
%%script C:\Users\Jan Catherine\AppData\Local\Programs\Python\Python38\python.exe
import sys
print(sys.version)

3.8.10 (tags/v3.8.10:3d8993a, May  3 2021, 11:48:03) [MSC v.1928 64 bit (AMD64)]


In [2]:
pwd

'C:\\Users\\Jan Catherine\\Documents\\CMSC Notes\\CMSC 190 Part 2\\Codes\\CNN'

In [3]:
import os
os.chdir('../../Data_Collection_02/02_Preprocessing/05_CNN')

In [4]:
#pip install tensorflow

In [5]:
#importing libraries
"""
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
"""

'\nfrom keras.models import Sequential\nfrom keras.layers import Dense, Dropout, Activation, Flatten\nfrom keras.layers import Convolution2D, MaxPooling2D\nfrom sklearn.model_selection import train_test_split\nimport pandas as pd\nimport numpy as np\n'

In [6]:
import matplotlib.pyplot as plt
import os
import re
import shutil
import string
import tensorflow as tf
from tensorflow.keras import regularizers

from tensorflow.keras import layers
from tensorflow.keras import losses

from collections import Counter

import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

from tensorflow.keras import preprocessing
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.losses import BinaryCrossentropy


from sklearn.model_selection import KFold

#import pydot

In [7]:
dataset = pd.read_csv('01_Title_Train.csv', encoding='latin-1')
testdata = pd.read_csv('01_Title_Test.csv', encoding='latin-1')

train_data = dataset
test_data = testdata

#Oversampling
count_class_0, count_class_1 = train_data.Fake.value_counts()

df_class_0 = train_data[train_data['Fake'] == 0]
df_class_1 = train_data[train_data['Fake'] == 1]

df_class_1_over = df_class_1.sample(count_class_0, replace=True)
df_test_over = pd.concat([df_class_0, df_class_1_over], axis=0)

df_test_over = df_test_over.reset_index(drop=True)
print('Random over-sampling:')
print(df_test_over.Fake.value_counts())

train_data = df_test_over
#train_data['Title'] = df_test_over['Title']
#train_data['Fake'] = df_test_over['Fake']

print(train_data.Fake.value_counts())

train_data.dropna(axis=0, how ='any', inplace=True)
train_data['Num_words_title'] = train_data['Title'].apply(lambda x:len(str(x).split()))
#train_data['Num_words_desc'] = train_data['Description'].apply(lambda x:len(str(x).split()))
print('-------Train data--------')
print(train_data['Fake'].value_counts())
print(len(train_data))
print('-------------------------')

max_train_title_length  = train_data['Num_words_title'].max()
#max_train_desc_length  = train_data['Num_words_desc'].max()


test_data.dropna(axis = 0, how ='any',inplace=True) 
test_data['Num_words_title'] = test_data['Title'].apply(lambda x:len(str(x).split())) 

max_test_title_length  = test_data['Num_words_title'].max()

print('-------Test data--------')
print(test_data['Fake'].value_counts())
print(len(test_data))
print('-------------------------')

print('Train Max Length :'+str(max_train_title_length))
print('Test Max Sentence Length :'+str(max_test_title_length))


Random over-sampling:
0    5760
1    5760
Name: Fake, dtype: int64
0    5760
1    5760
Name: Fake, dtype: int64
-------Train data--------
0    5760
1    5760
Name: Fake, dtype: int64
11520
-------------------------
-------Test data--------
0    1428
1     184
Name: Fake, dtype: int64
1612
-------------------------
Train Max Length :20
Test Max Sentence Length :17


In [8]:
train_data.head()

,Title,Fake,Num_words_title
0,key arsenal rural India pandemic fight,0,6
1,Coronavirus banks : Implications leaders,0,5
2,Nail Salons Massage Parlors Tattoo Studios,0,6
3,Risk SARSCoV2 transmission aerosols rational,0,5
4,House prices Spain : COVID19 affected idealista,0,7


In [9]:
num_words = 20000


tokenizer = Tokenizer(num_words=num_words, oov_token="unk")
tokenizer.fit_on_texts(train_data['Title'].tolist())

print(str(tokenizer.texts_to_sequences(['5G Global bill airfare'])))

[[5, 57, 21, 1]]


In [10]:
"""word_index = tokenizer.word_index
vocab_size = len(word_index) + 1
print(f'Vocabulary Size : {vocab_size}')
"""

"word_index = tokenizer.word_index\nvocab_size = len(word_index) + 1\nprint(f'Vocabulary Size : {vocab_size}')\n"

In [11]:
X_train, X_valid, y_train, y_valid = train_test_split(train_data['Title'].tolist(),\
                                                      train_data['Fake'].tolist(),\
                                                      test_size=0.2,\
                                                      stratify = train_data['Fake'].tolist(),\
                                                      random_state=0)

print('Train data len:'+str(len(X_train)))
print('Class distribution'+str(Counter(y_train)))
print('Valid data len:'+str(len(X_valid)))
print('Class distribution'+ str(Counter(y_valid)))

x_train = np.array( tokenizer.texts_to_sequences(X_train) )
x_valid = np.array( tokenizer.texts_to_sequences(X_valid) )
x_test  = np.array( tokenizer.texts_to_sequences(test_data['Title'].tolist()) )

x_train = pad_sequences(x_train, padding='post', maxlen=90)
x_valid = pad_sequences(x_valid, padding='post', maxlen=90)
x_test = pad_sequences(x_test, padding='post', maxlen=90)

print(x_train[0])

le = LabelEncoder()

train_labels = le.fit_transform(y_train)
train_labels = np.asarray( tf.keras.utils.to_categorical(train_labels, 2))
#print(train_labels)
valid_labels = le.transform(y_valid)
valid_labels = np.asarray( tf.keras.utils.to_categorical(valid_labels, 2))


test_labels = le.transform(test_data['Fake'].tolist())
test_labels = np.asarray(tf.keras.utils.to_categorical(test_labels, 2))
list(le.classes_)


train_ds = tf.data.Dataset.from_tensor_slices((x_train,train_labels))
valid_ds = tf.data.Dataset.from_tensor_slices((x_valid,valid_labels))
test_ds = tf.data.Dataset.from_tensor_slices((x_test,test_labels))

Train data len:9216
Class distributionCounter({0: 4608, 1: 4608})
Valid data len:2304
Class distributionCounter({1: 1152, 0: 1152})
[6186  260  520    4   25   61 2647    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0]


C:\Users\Jan Catherine\AppData\Local\Temp\ipykernel_7328\740212987.py:12: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  x_train = np.array( tokenizer.texts_to_sequences(X_train) )
C:\Users\Jan Catherine\AppData\Local\Temp\ipykernel_7328\740212987.py:13: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  x_valid = np.array( tokenizer.texts_to_sequences(X_valid) )
C:\Users\Jan Catherine\AppData\Local\Temp\ipykernel_7328\740212987.py:14: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-o

In [12]:
test_data.head()

,Title,Fake,Num_words_title
0,Coronavirus fact check : recycled air planes,0,7
1,Coronavirus Italy travel advice : safe travel ...,0,9
2,Wayne Rooney pays tribute wife Coleen wedding ...,0,11
3,Trump blame game China disguise massive COVID1...,0,10
4,COVID19 Testing Locations,0,3


In [13]:
"""print(y_train[:10])
train_labels = le.fit_transform(y_train)
print('Text to number')
print(train_labels[:10])
train_labels = np.asarray( tf.keras.utils.to_categorical(train_labels))
print('Number to category')
print(train_labels[:10])
"""

"print(y_train[:10])\ntrain_labels = le.fit_transform(y_train)\nprint('Text to number')\nprint(train_labels[:10])\ntrain_labels = np.asarray( tf.keras.utils.to_categorical(train_labels))\nprint('Number to category')\nprint(train_labels[:10])\n"

In [14]:
count =0
print('======Train dataset ====')
for value,label in train_ds:
    count += 1
    print(value,label)
    if count==3:
        break
count =0
print('======Validation dataset ====')
for value,label in valid_ds:
    count += 1
    print(value,label)
    if count==3:
        break
count = 0
print('======Test dataset ====')
for value,label in test_ds:
    count += 1
    print(value,label)
    if count==3:
        break  


======Train dataset ====
tf.Tensor(
[6186  260  520    4   25   61 2647    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0], shape=(90,), dtype=int32) tf.Tensor([1. 0.], shape=(2,), dtype=float32)
tf.Tensor(
[ 95 282 213 292 824   7   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0], shape=(90,), dtype=int32) tf.Tensor([1. 0.], shape=(2,)

In [15]:
#CNN MODEL

max_features = 11718
embedding_dim = 64 #same as URLNet
sequence_length = 90

# Define per-fold score containers
acc_per_fold = []
loss_per_fold = []

inputs = np.concatenate((x_train,x_valid), axis=0)
targets = np.concatenate((train_labels, valid_labels), axis=0)

#regularizer prevents overfitting
# Define the K-fold Cross Validator
kfold = KFold(n_splits=5, shuffle=True)

# K-fold Cross Validation model evaluation
fold_no = 1
for train, test in kfold.split(inputs, targets):
    
    
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Embedding(max_features +1, embedding_dim, input_length=sequence_length,\
                                        embeddings_regularizer = regularizers.l2(0.0005)))                                    

    model.add(tf.keras.layers.Conv1D(128,3, activation='relu',\
                                     kernel_regularizer = regularizers.l2(0.0005),\
                                     bias_regularizer = regularizers.l2(0.0005)))                               


    model.add(tf.keras.layers.GlobalMaxPooling1D())

    model.add(tf.keras.layers.Dropout(0.5)) #to reduce overfitting

    #final classification, 2 classes
    model.add(tf.keras.layers.Dense(2, activation='sigmoid',\
                                    kernel_regularizer=regularizers.l2(0.001),\
                                    bias_regularizer=regularizers.l2(0.001),))




    model.summary()
    model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=False), optimizer='Nadam', metrics=["CategoricalAccuracy"])
    
    epochs = 100
    # Fit the model using the train and test datasets.
    #history = model.fit(x_train, train_labels,validation_data= (x_test,test_labels),epochs=epochs )
    history = model.fit(inputs[train], targets[train],
                        epochs= epochs ,
                        verbose=1)
    # Generate generalization metrics
    scores = model.evaluate(inputs[test], targets[test], verbose=0)
    print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
    acc_per_fold.append(scores[1] * 100)
    loss_per_fold.append(scores[0])

    # Increase fold number
    fold_no = fold_no + 1

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 90, 64)            750016    
                                                                 
 conv1d (Conv1D)             (None, 88, 128)           24704     
                                                                 
 global_max_pooling1d (Glob  (None, 128)               0         
 alMaxPooling1D)                                                 
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense (Dense)               (None, 2)                 258       
                                                                 
Total params: 774978 (2.96 MB)
Trainable params: 774978 (2.96 MB)
Non-trainable params: 0 (0.00 Byte)
____________________

KeyboardInterrupt: 

In [ ]:
# == Provide average scores ==
print('------------------------------------------------------------------------')
print('Score per fold')
for i in range(0, len(acc_per_fold)):
  print('------------------------------------------------------------------------')
  print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {acc_per_fold[i]}%')
print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
print(f'> Loss: {np.mean(loss_per_fold)}')
print('------------------------------------------------------------------------')

In [17]:
history.history

{'loss': [0.5477079153060913,
  0.22543710470199585,
  0.1701410859823227,
  0.15694795548915863,
  0.15227153897285461,
  0.1496129035949707,
  0.15003007650375366,
  0.1484369933605194,
  0.14846058189868927,
  0.14879468083381653,
  0.14865358173847198,
  0.14683188498020172,
  0.14828363060951233,
  0.1484348177909851,
  0.1464097648859024,
  0.14738918840885162,
  0.14583763480186462,
  0.14640407264232635,
  0.1456795632839203,
  0.14453111588954926,
  0.14725655317306519,
  0.14615312218666077,
  0.14414525032043457,
  0.14563769102096558,
  0.14506112039089203,
  0.14394523203372955,
  0.14446957409381866,
  0.14244969189167023,
  0.14285022020339966,
  0.1426951140165329,
  0.14380700886249542,
  0.14237645268440247,
  0.1410638988018036,
  0.1413307785987854,
  0.14065681397914886,
  0.14046092331409454,
  0.14100152254104614,
  0.1404103934764862,
  0.1391919106245041,
  0.13926194608211517,
  0.1405961513519287,
  0.13847416639328003,
  0.1386929303407669,
  0.1392384469509

In [18]:
"""plt.plot(history.history['loss'], label=' training data')
plt.plot(history.history['val_loss'], label='validation data')
plt.title('Loss for Text Classification')
plt.ylabel('Loss value')
plt.xlabel('No. epoch')
plt.legend(loc="upper left")
plt.show()
"""

'plt.plot(history.history[\'loss\'], label=\' training data\')\nplt.plot(history.history[\'val_loss\'], label=\'validation data\')\nplt.title(\'Loss for Text Classification\')\nplt.ylabel(\'Loss value\')\nplt.xlabel(\'No. epoch\')\nplt.legend(loc="upper left")\nplt.show()\n'

In [19]:
"""plt.plot(history.history['categorical_accuracy'], label=' (training data)')
plt.plot(history.history['val_categorical_accuracy'], label='CategoricalCrossentropy (validation data)')
plt.title('CategoricalAccuracy for Text Classification')
plt.ylabel('CategoricalAccuracy value')
plt.xlabel('No. epoch')
plt.legend(loc="upper left")
plt.show()
"""

'plt.plot(history.history[\'categorical_accuracy\'], label=\' (training data)\')\nplt.plot(history.history[\'val_categorical_accuracy\'], label=\'CategoricalCrossentropy (validation data)\')\nplt.title(\'CategoricalAccuracy for Text Classification\')\nplt.ylabel(\'CategoricalAccuracy value\')\nplt.xlabel(\'No. epoch\')\nplt.legend(loc="upper left")\nplt.show()\n'

In [33]:
for layer in model.layers:
    layer._name = layer.name + str("_1")

In [34]:
model.save('C:\\Users\\Jan Catherine\\Documents\\CMSC Notes\\CMSC 190 Part 2\\Codes\\CNN\\Title') 
json_string1 = tokenizer.to_json()

INFO:tensorflow:Assets written to: C:\Users\Jan Catherine\Documents\CMSC Notes\CMSC 190 Part 2\Codes\CNN\Title\assets


INFO:tensorflow:Assets written to: C:\Users\Jan Catherine\Documents\CMSC Notes\CMSC 190 Part 2\Codes\CNN\Title\assets


In [35]:
import json
with open('C:\\Users\\Jan Catherine\\Documents\\CMSC Notes\\CMSC 190 Part 2\\Codes\\CNN\\Title\\Title.json', 'w') as outfile1:
    json.dump(json_string1, outfile1)

In [36]:
#reload model

new_model = tf.keras.models.load_model('C:\\Users\\Jan Catherine\\Documents\\CMSC Notes\\CMSC 190 Part 2\\Codes\\CNN\\Title')
new_model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4_1 (Embedding)   (None, 90, 64)            750016    
                                                                 
 conv1d_4_1 (Conv1D)         (None, 88, 128)           24704     
                                                                 
 global_max_pooling1d_4_1 (  (None, 128)               0         
 GlobalMaxPooling1D)                                             
                                                                 
 dropout_4_1 (Dropout)       (None, 128)               0         
                                                                 
 dense_4_1 (Dense)           (None, 2)                 258       
                                                                 
Total params: 774978 (2.96 MB)
Trainable params: 774978 (2.96 MB)
Non-trainable params: 0 (0.00 Byte)
__________________

In [37]:
with open('C:\\Users\\Jan Catherine\\Documents\\CMSC Notes\\CMSC 190 Part 2\\Codes\\CNN\\Title\\Title.json') as json_file1:
    json_string1 = json.load(json_file1)
tokenizer1 = tf.keras.preprocessing.text.tokenizer_from_json(json_string1)

In [38]:
x_test  = np.array( tokenizer1.texts_to_sequences(test_data['Title'].tolist()) )
x_test = pad_sequences(x_test, padding='post', maxlen=90)

C:\Users\Jan Catherine\AppData\Local\Temp\ipykernel_17704\3760519126.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  x_test  = np.array( tokenizer1.texts_to_sequences(test_data['Title'].tolist()) )


In [39]:
# Generate predictions (probabilities -- the output of the last layer)
# on test  data using `predict`
print("Generate predictions for all samples")
predictions = new_model.predict(x_test)
print(predictions)
predict_results = predictions.argmax(axis=1)

Generate predictions for all samples
51/51 [==============================] - 0s 3ms/step
[[0.9986129  0.00138709]
 [0.99433863 0.00566135]
 [0.99058545 0.00941452]
 ...
 [0.00372173 0.9962783 ]
 [0.9556978  0.04430221]
 [0.9832606  0.01673941]]


In [40]:
test_data['pred_fake']= predict_results
test_data['pred_fake'] = np.where((test_data.pred_fake == '0'),0,test_data.pred_fake)
test_data['pred_fake'] = np.where((test_data.pred_fake == '1'),1,test_data.pred_fake)

In [41]:
print(type(test_data['Fake']))
test_data['Fake'].head()


<class 'pandas.core.series.Series'>


0    0
1    0
2    0
3    0
4    0
Name: Fake, dtype: int64

In [42]:
print(type(test_data['pred_fake']))
test_data['pred_fake'].head()

<class 'pandas.core.series.Series'>


0    0
1    0
2    0
3    0
4    0
Name: pred_fake, dtype: int64

In [43]:
labels = [0, 1]
    
print(classification_report(test_data['Fake'].tolist(),test_data['pred_fake'].tolist(),labels=labels))

              precision    recall  f1-score   support

           0       0.96      0.97      0.97      1428
           1       0.76      0.71      0.73       184

    accuracy                           0.94      1612
   macro avg       0.86      0.84      0.85      1612
weighted avg       0.94      0.94      0.94      1612



In [ ]:
"""precisionscore = precision_score(y_test, new_model.predict(x_test))
accuracyscore = accuracy_score(y_test, new_model.predict(x_test))
recallscore = recall_score(y_test, new_model.predict(x_test))
f1score = f1_score(y_test, new_model.predict(x_test))
cm = confusion_matrix(y_test, new_model.predict(x_test))

print(precisionscore, accuracyscore, recallscore, f1score, cm)
"""